**OBJECTIVE:** Document my thought process as I develop the function for the `/v1/country-data/` endpoint

git push --set-upstream origin feature/country-endpoint-alexgabriellafaith

In [4]:
import pandas as pd

# Read the Data

In [5]:
df_assessments = pd.read_excel("./data/TPI ASCOR data - 13012025/ASCOR_assessments_results.xlsx")

df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])
df_assessments['Publication date'] = pd.to_datetime(df_assessments['Publication date'])

/var/folders/m5/fkxp7dk132s2dqm5fccfhy9h0000gn/T/ipykernel_13986/2544544804.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])


## Test how to filter the df:

In [9]:
selected_row = (
    (df_assessments["Country"] == country) &
    (df_assessments['Assessment date'].dt.year == assessment_year)
)

In [10]:
country = 'United Kingdom'
assessment_year = 2023

# This returns a pd.Series of Trues and Falses
selected_country = df_assessments["Country"] == country
selected_year    = df_assessments['Assessment date'].dt.year == assessment_year


df_assessments[selected_country & selected_year]

,Id,Assessment date,Publication date,Country Id,Country,area EP.1,indicator EP.1.a,indicator EP.1.b,indicator EP.1.c,area EP.2,...,year metric CP.4.d.i,year metric CP.4.e.i,year metric CP.6.a.i,year metric CF.1.a.i,year metric CF.1.b.i,year metric CF.4.i,year metric CF.4.ii,year metric CF.4.iii,year metric CF.4.iv,Notes
22,86,2023-10-31,2023-01-12,22,United Kingdom,Partial,Yes,No,No,Partial,...,2021.0,2022.0,2021,2020.0,2021.0,2023,2023,2023,2023,Assessment results are based on in-depth polic...


# The function I am designing 

In [11]:
def get_country_data(country: str, assessment_year: int):

    selected_row = (
        (df_assessments["Country"] == country) &
        (df_assessments['Assessment date'].dt.year == assessment_year)
    )

    # Filter the data
    data = df_assessments[selected_row]

    # Selected and filter columns
    area_columns = [col for col in df_assessments.columns if col.startswith("area")]
    data = data[area_columns]
    
    # JSON does not allow for NaN or NULL. 
    # The equivalent is just to leave an empty string instead
    data = data.fillna('')

    #Rename columns
    data['country'] = country
    data['assessment_year'] = assessment_year

    remap_area_column_names = {
        col: col.replace('area ', '')
        for col in area_columns
    }

    data = data.rename(columns=remap_area_column_names)

    # Grab just the first element (there should only be one anyway)
    # and return it as a dictionary
    return data.iloc[0].to_dict()

Test how the function behaves:

In [12]:
get_country_data('Italy', 2024)

{'EP.1': 'Partial',
 'EP.2': 'Partial',
 'EP.3': 'Partial',
 'CP.1': 'No',
 'CP.2': 'Partial',
 'CP.3': 'Partial',
 'CP.4': 'Partial',
 'CP.5': 'Partial',
 'CP.6': 'Partial',
 'CF.1': 'No',
 'CF.2': 'Exempt',
 'CF.3': 'Partial',
 'CF.4': '',
 'country': 'Italy',
 'assessment_year': 2024}

# Testing out Pydantic Models

In [13]:
from pydantic import BaseModel

class CountryData(BaseModel):
    country: str
    assesment_year: int

In the end, we want our API to produce an **instance** of the CountryData object like this:

In [14]:
output = CountryData(country="United Kingdom", assesment_year=2024)

output

CountryData(country='United Kingdom', assesment_year=2024)

In reality, all I have (usually) is a dictionary or a list that looks like this:

In [15]:
output_dict = {"country": "United Kingdom", "assesment_year":2024}

output_dict

{'country': 'United Kingdom', 'assesment_year': 2024}

The ** operator allows to pass a dictionary to a class or a function so that each key becomes a parameter/argument: 

In [16]:
CountryData(**output_dict)

CountryData(country='United Kingdom', assesment_year=2024)

# Start working on the deeply hierarchical structure

Eventually, I want to serve data like this:

```json
{
    "pillars": [
    {
        "name": "EP",
        "areas": [
            {
                "name": "EP.1",
                "assessment": "Partial",
                "indicators": [
                    {
                        "name": "EP.1.a",
                        "assessment": "Yes",
                        "metrics": ""
                    },
                    ...
                    {
                        "name": "EP.2.1",
                        "assessment": "Yes",
                        "metrics": {
                            "name": "EP.2.a.1",
                            "value": "-25%"
                        }
                    }
                ]
            }
        ]
    },
    {
        "name": "CP",
        "areas": [
            {
                "name": "CP.1",
                "assessment": "Partial",
                "indicators": [
                    ...
                ]
            },
            {
                ...
            }
        ]
    },
    {
        "name": "CF",
        "areas": [
            {
                "name": "CF.1",
                "assessment": "Partial",
                "indicators": [
                    ...
                ]
            },
            {
                ...
            }
        ]
    }
    ]
}
```


## Focus on just the Metrics part

In [17]:
# This is the data model for Metrics
class Metric(BaseModel):
    name: str
    value: str

Before I actually write the code to the API, let me see how I'd have to filter the dataframe:

In [18]:
# Filter for just the metrics columns
selected_columns = [col for col in df_assessments.columns if col.startswith('metric')]
selected_columns

['metric EP.2.a.i',
 'metric EP.2.b.i',
 'metric EP.2.c.i',
 'metric EP.2.d.i',
 'metric EP.3.a.i',
 'metric CP.2.b.i',
 'metric CP.2.c.i',
 'metric CP.3.a.i',
 'metric CP.3.b.i',
 'metric CP.3.c.i',
 'metric CP.3.d.i',
 'metric CP.3.d.ii',
 'metric CP.4.b.i',
 'metric CP.4.d.i',
 'metric CP.4.e.i',
 'metric CP.6.a.i',
 'metric CF.1.a.i',
 'metric CF.1.b.i',
 'metric CF.4.i',
 'metric CF.4.ii',
 'metric CF.4.iii',
 'metric CF.4.iv']

Filter the data point to include only those columns:

In [28]:
# Get a random data point
data = df_assessments.iloc[0]

data[selected_columns]

metric EP.2.a.i                            '-25%
metric EP.2.b.i      No or unsuitable disclosure
metric EP.2.c.i                              62%
metric EP.2.d.i                             822%
metric EP.3.a.i                             2050
metric CP.2.b.i                              28%
metric CP.2.c.i                  US$ 10.64/tCO₂e
metric CP.3.a.i      No or unsuitable disclosure
metric CP.3.b.i                            0.34%
metric CP.3.c.i                            0.79%
metric CP.3.d.i                            0.26%
metric CP.3.d.ii                           1.72%
metric CP.4.b.i     4.28 MJ/US$ PPP-adjusted GDP
metric CP.4.d.i                              27%
metric CP.4.e.i                              22%
metric CP.6.a.i                             0.94
metric CF.1.a.i                            0.02%
metric CF.1.b.i                            0.02%
metric CF.4.i            43.5 MW/US$ billion GDP
metric CF.4.ii          88.42 MW/US$ billion GDP
metric CF.4.iii     

How would the Metric object be constructed?

In [20]:
Metric(name="metric EP.2.a.i", value="-25%")

Metric(name='metric EP.2.a.i', value='-25%')

In [21]:
data_as_dict = data[selected_columns].to_dict()

list_metrics = []
for name, value in data_as_dict.items():
    individual_metric = Metric(name=name, value=value)
    list_metrics.append(individual_metric)

list_metrics

[Metric(name='metric EP.2.a.i', value="'-25%"),
 Metric(name='metric EP.2.b.i', value='No or unsuitable disclosure'),
 Metric(name='metric EP.2.c.i', value='62%'),
 Metric(name='metric EP.2.d.i', value='822%'),
 Metric(name='metric EP.3.a.i', value='2050'),
 Metric(name='metric CP.2.b.i', value='28%'),
 Metric(name='metric CP.2.c.i', value='US$ 10.64/tCO₂e'),
 Metric(name='metric CP.3.a.i', value='No or unsuitable disclosure'),
 Metric(name='metric CP.3.b.i', value='0.34%'),
 Metric(name='metric CP.3.c.i', value='0.79%'),
 Metric(name='metric CP.3.d.i', value='0.26%'),
 Metric(name='metric CP.3.d.ii', value='1.72%'),
 Metric(name='metric CP.4.b.i', value='4.28 MJ/US$ PPP-adjusted GDP'),
 Metric(name='metric CP.4.d.i', value='27%'),
 Metric(name='metric CP.4.e.i', value='22%'),
 Metric(name='metric CP.6.a.i', value='0.94'),
 Metric(name='metric CF.1.a.i', value='0.02%'),
 Metric(name='metric CF.1.b.i', value='0.02%'),
 Metric(name='metric CF.4.i', value='43.5 MW/US$ billion GDP'),
 Metr

## Test the whole new function 

In [22]:
def get_country_metrics(country: str, assessment_year: int):

    selected_row = (
        (df_assessments["Country"] == country) &
        (df_assessments['Assessment date'].dt.year == assessment_year)
    )

    # Filter the data
    data = df_assessments[selected_row]

    if data.empty:
        raise HTTPException(status_code=404, 
                            detail=f"There is no data for country: {country} and year: {assessment_year}")

    # Select just the metrics
    metric_columns = [col for col in df_assessments.columns 
                      if col.startswith('metric')]
    data = data[metric_columns]

    # JSON does not allow for NaN or NULL. 
    # The equivalent is just to leave an empty string instead
    data = data.fillna('')

    remap_area_column_names = {
        col: col.replace('metric ', '')
        for col in metric_columns
    }

    data = data.rename(columns=remap_area_column_names)

    data_as_dict = data.iloc[0].to_dict()

    list_metrics = []
    for name, value in data_as_dict.items():
        individual_metric = Metric(name=name, value=value)
        list_metrics.append(individual_metric)
    # Grab just the first element (there should only be one anyway)
    # and return it as a dictionary
    return list_metrics


In [23]:
get_country_metrics('Italy', 2024)

[Metric(name='EP.2.a.i', value='-30%'),
 Metric(name='EP.2.b.i', value='0%'),
 Metric(name='EP.2.c.i', value='86%'),
 Metric(name='EP.2.d.i', value='317%'),
 Metric(name='EP.3.a.i', value='2050'),
 Metric(name='CP.2.b.i', value='33%'),
 Metric(name='CP.2.c.i', value='89.88 US$/tCO2e'),
 Metric(name='CP.3.a.i', value='2025'),
 Metric(name='CP.3.b.i', value='0.42%'),
 Metric(name='CP.3.c.i', value='0.00%'),
 Metric(name='CP.3.d.i', value='0.08%'),
 Metric(name='CP.3.d.ii', value='0.03%'),
 Metric(name='CP.4.b.i', value='2.30 MJ/PPP-adjusted GDP'),
 Metric(name='CP.4.d.i', value='46%'),
 Metric(name='CP.4.e.i', value='22%'),
 Metric(name='CP.6.a.i', value='83%'),
 Metric(name='CF.1.a.i', value='0.07%'),
 Metric(name='CF.1.b.i', value='0.13%'),
 Metric(name='CF.4.i', value='2.24 MW/US$ billion GDP'),
 Metric(name='CF.4.ii', value='20.56 MW/US$ billion GDP'),
 Metric(name='CF.4.iii', value='0.00 MW/US$ billion GDP'),
 Metric(name='CF.4.iv', value='0.12 MW/US$ billion GDP')]

## How to group together metrics with their relevant indicators?

In [ ]:
indicator_columns = [col for col in df_assessments.columns if col.startswith('indicator ')]

for indicator in indicator_columns:
    get row of indicator in df_assessments
    then store the Country and Assessment date of this indicators row 
    indicator = text.split("indicator ")[-1]
    # find the metric associated with this specific indicator
    metric = [col for col in df_assessments.columns if col.startswith('metric') then metric = metric_name.rsplit(".", 1)[0]
    if metric==indicator:
        get_country_metrics(country of indicator: str, assessment_year indicator: int):
        
              

['indicator EP.1.a',
 'indicator EP.1.b',
 'indicator EP.1.c',
 'indicator EP.2.a',
 'indicator EP.2.b',
 'indicator EP.2.c',
 'indicator EP.2.d',
 'indicator EP.3.a',
 'indicator EP.3.b',
 'indicator EP.3.c',
 'indicator CP.1.a',
 'indicator CP.1.b',
 'indicator CP.2.a',
 'indicator CP.2.b',
 'indicator CP.2.c',
 'indicator CP.3.a',
 'indicator CP.3.b',
 'indicator CP.3.c',
 'indicator CP.3.d',
 'indicator CP.4.a',
 'indicator CP.4.b',
 'indicator CP.4.c',
 'indicator CP.4.d',
 'indicator CP.4.e',
 'indicator CP.5.a',
 'indicator CP.5.b',
 'indicator CP.5.c',
 'indicator CP.5.d',
 'indicator CP.5.e',
 'indicator CP.6.a',
 'indicator CP.6.b',
 'indicator CP.6.c',
 'indicator CP.6.d',
 'indicator CF.1.a',
 'indicator CF.1.b',
 'indicator CF.2.a',
 'indicator CF.2.b',
 'indicator CF.3.a',
 'indicator CF.3.b']

# Formative 2 Work

In [ ]:
df_assessments = pd.read_excel("./data/TPI ASCOR data - 13012025/ASCOR_assessments_results.xlsx")
df_assessments
data = data.iloc[0]

,Id,Assessment date,Publication date,Country Id,Country,area EP.1,indicator EP.1.a,indicator EP.1.b,indicator EP.1.c,area EP.2,...,year metric CP.4.d.i,year metric CP.4.e.i,year metric CP.6.a.i,year metric CF.1.a.i,year metric CF.1.b.i,year metric CF.4.i,year metric CF.4.ii,year metric CF.4.iii,year metric CF.4.iv,Notes
0,82,31/10/2023,01/12/2023,8,Australia,Partial,Yes,No,No,Partial,...,2021.0,2022.0,2021,2020.0,2023.0,2023,2023,2023,2023,Assessment results are based on in-depth polic...
1,88,31/10/2023,01/12/2023,25,Bangladesh,Partial,No,No,Yes,Partial,...,NaN,NaN,2021,NaN,NaN,2023,2023,2023,2023,Assessment results are based on in-depth polic...
2,96,31/10/2023,01/12/2023,24,Barbados,Yes,Yes,No data,Yes,Partial,...,2015.0,2022.0,2021,NaN,NaN,2023,2023,2023,2023,Assessment results are based on in-depth polic...
3,91,31/10/2023,01/12/2023,18,Brazil,Partial,Yes,No,No,Partial,...,2020.0,2022.0,2021,NaN,NaN,2023,2023,2023,2023,Assessment results are based on in-depth polic...
4,72,31/10/2023,01/12/2023,14,Canada,Partial,Yes,No,No,Partial,...,2021.0,2022.0,2021,2020.0,2021.0,2023,2023,2023,2023,Assessment results are based on in-depth polic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,54,23/08/2024,01/11/2024,57,Türkiye,No,No,No,No,Partial,...,2023.0,2023.0,2022,NaN,NaN,2024,2024,2024,2024,Assessment results are based on in-depth polic...
91,55,23/08/2024,01/11/2024,61,United Arab Emirates,No,No,No,No,Partial,...,2022.0,2023.0,2022,NaN,NaN,2024,2024,2024,2024,Assessment results are based on in-depth polic...
92,22,23/08/2024,01/11/2024,22,United Kingdom,Partial,Yes,No,No,Partial,...,2023.0,2023.0,2022,2020.0,2021.0,2024,2024,2024,2024,Assessment results are based on in-depth polic...
93,1,23/08/2024,01/11/2024,1,United States,Partial,Yes,No,No,Partial,...,2023.0,2023.0,2022,2020.0,2023.0,2024,2024,2024,2024,Assessment results are based on in-depth polic...


In [50]:
indicator_columns = [col for col in df_assessments.columns if col.startswith('indicator ')]

# Initialize an empty list to store results
indicator_metrics = []

for indicator_col in indicator_columns:
    # Get the row that contains the indicator
    indicator_name = indicator_col.split("indicator ")[-1]  # Extract the name of the indicator
    
    # Filter the data for the indicator column (assuming one row for each country and year)
    selected_row = df_assessments[df_assessments[indicator_col].notna()]

    if selected_row.empty:
        continue  # Skip if no row is found for this indicator
    # Get Country and Assessment date from the row
    country = selected_row.iloc[0]["Country"]
    assessment_year = selected_row.iloc[0]["Assessment date"].year

    # Now, find the corresponding metric for this indicator
    metric_columns = [col for col in df_assessments.columns if col.startswith('metric')]
    for metric_col in metric_columns:
        # Get the base name of the metric (remove the last section after the final dot)

        metric_name = metric_col.replace('metric ', '')
        metric_name = ".".join(metric_name.split(".")[:3])
        if metric_name == indicator_name:
            print(metric_name)
            print(indicator_name)
            # If the metric matches the indicator, extract the metric value
            metric_value = selected_row.iloc[0][metric_col]
            
            # Append the indicator and its metric to the results list
            indicator_metrics.append({
                "indicator": indicator_name,
                "assessment": selected_row.iloc[0][indicator_col],
                "country": country,
                "assessment_year": assessment_year,
                "metric": {
                    "name": metric_name,
                    "value": metric_value
                }
            })



EP.2.a
EP.2.a
EP.2.b
EP.2.b
EP.2.c
EP.2.c
EP.2.d
EP.2.d
EP.3.a
EP.3.a
CP.2.b
CP.2.b
CP.2.c
CP.2.c
CP.3.a
CP.3.a
CP.3.b
CP.3.b
CP.3.c
CP.3.c
CP.3.d
CP.3.d
CP.3.d
CP.3.d
CP.4.b
CP.4.b
CP.4.d
CP.4.d
CP.4.e
CP.4.e
CP.6.a
CP.6.a
CF.1.a
CF.1.a
CF.1.b
CF.1.b
